In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import matplotlib.pyplot as plt
import model.cell as cell
import model.spec as spec
import numpy as np


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [2]:
np.shape(train_images)

(50000, 32, 32, 3)

In [37]:

cf = spec.Spec.get_configuration_space().sample_configuration()
s = spec.Spec(cf)
while s.valid_spec == False:
    cf = spec.Spec.get_configuration_space().sample_configuration()
    s = spec.Spec(cf)

c = cell.Cell(s, 64)

print(s.matrix)
print(s.ops)
c.compute_vertex_channels(128, 128, s.matrix)

[[0 1 0 0 0 1]
 [0 0 1 0 0 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]
['input', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'output']


[128, 64, 64, 64, 64, 128]

In [38]:
i = tf.keras.Input(shape=(32, 32, 3))
o = c.build(i)
o = tf.keras.layers.Flatten()(o)
o = tf.keras.layers.Dense(64, activation='relu')(o)
o = tf.keras.layers.Dense(10)(o)

model = tf.keras.Model(i, o, name="sample_cell")
model.summary()

for l in model.layers:
    print("%s %s" % (l.name, str(l.output_shape)))


Model: "sample_cell"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 32, 32, 32)   96          input_17[0][0]                   
__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, 32, 32, 32)   128         conv2d_56[0][0]                  
__________________________________________________________________________________________________
re_lu_56 (ReLU)                 (None, 32, 32, 32)   0           batch_normalization_56[0][0]     
________________________________________________________________________________________

In [39]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 10s 200us/sample - loss: 2.3707 - accuracy: 0.1042 - val_loss: 2.3027 - val_accuracy: 0.1000
Epoch 2/5
50000/50000 [==============================] - 9s 189us/sample - loss: 2.3031 - accuracy: 0.0980 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/5
50000/50000 [==============================] - 9s 188us/sample - loss: 2.3028 - accuracy: 0.0987 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 4/5
50000/50000 [==============================] - 9s 187us/sample - loss: 2.3028 - accuracy: 0.0961 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 5/5
50000/50000 [==============================] - 9s 188us/sample - loss: 2.3028 - accuracy: 0.0983 - val_loss: 2.3026 - val_accuracy: 0.1000
